In [ ]:
import nltk
import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Lambda
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential

text = 'Deep learning is a subset of machine learning that focuses on utilizing neural networks to perform tasks such as classification, regression, and representation learning. The field takes inspiration from biological neuroscience and is centered around stacking artificial neurons into layers and "training" them to process data. The adjective "deep" refers to the use of multiple layers (ranging from three to several hundred or thousands) in the network. Methods used can be either supervised, semi-supervised or unsupervised. Some common deep learning network architectures include fully connected networks, deep belief networks, recurrent neural networks, convolutional neural networks, generative adversarial networks, transformers, and neural radiance fields. These architectures have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board gamef programs, where they have produced results comparable to and in some cases surpassing human expert performance. Early forms of neural networks were inspired by information processing and distributed communication nodes in biological systems, particularly the human brain. However, current neural networks do not intend to model the brain function of organisms, and are generally seen as low-quality models for that purpose.'


# Splitting the text into sentences

sentences = text.split('.')
sentences = [sentence.lower() for sentence in sentences]
sentences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
print(sequences)

idx2word = tokenizer.index_word
word2idx = tokenizer.word_index

print(idx2word, '\n')
print(word2idx)

vocab_size = len(tokenizer.word_index) + 1
emb_size = 100
context_size = 2

contexts = []
targets = []

for sequence in sequences:
    for i in range(context_size, len(sequence) - context_size):
        target = sequence[i]
        context = [sequence[i - 2], sequence[i - 1], sequence[i + 1], sequence[i + 2]]
        contexts.append(context)
        targets.append(target)

print(contexts, "\n")
print(targets)

# Printing features with target
for i in range(5):
    words = []
    target = idx2word.get(targets[i])
    for j in contexts[i]:
        words.append(idx2word.get(j))
    print(words," -> ", target)


X = np.array(contexts)
y = np.array(targets)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=emb_size, input_length=2 * context_size),
    Lambda(lambda x: tf.reduce_mean(x, axis=1)),
    Dense(256, activation='relu'),
    Dense(512, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50)

test_words = []

for idx in X_test[0]:
    test_words.append(idx2word.get(idx))

# Reshape X_test[0] to have a batch dimension
input_data = np.expand_dims(X_test[0], axis=0)

pred = model.predict(input_data)
pred = np.argmax(pred[0])

print("pred ", test_words, "\n=", idx2word.get(pred), "\n\n")